In [10]:
from sagemaker.s3 import S3Uploader

file_key = 'model.tar.gz'
model_artifact = S3Uploader.upload(file_key,'s3://my-trained-model-dbcnn/artifacts')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [1]:
import torch
print(torch.__version__)

2.0.0.post101


In [10]:
from sagemaker.image_uris import retrieve

deploy_instance_type = 'ml.t2.xlarge'

pytorch_inference_image_uri = retrieve('huggingface',
                                       region='us-east-2',
                                       version='4.6.1',
                                       instance_type=deploy_instance_type,
                                       base_framework_version='pytorch1.8.1',
                                       image_scope='inference')

In [11]:
print(pytorch_inference_image_uri)

763104351884.dkr.ecr.us-east-2.amazonaws.com/huggingface-pytorch-inference:1.8.1-transformers4.6.1-cpu-py36-ubuntu18.04


In [12]:
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker import get_execution_role

role = get_execution_role()
# hub = {
# 'HF_MODEL_ID':'distilbert-base-uncased-finetuned-sst-2-english',
# 'HF_TASK':'summarization'
# }
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data="s3://my-trained-model-dbcnn-3/model.tar.gz", # path to your trained sagemaker model
   image_uri=pytorch_inference_image_uri,
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.6.1", # transformers version used
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [13]:
# import os
# os.environ["TASK"] = "summarization"

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1, 
   instance_type="ml.t2.xlarge"
)

-----!

In [14]:
predictor.endpoint_name

'huggingface-pytorch-inference-2023-12-22-09-50-53-883'

In [26]:
predictor.endpoint_name

'huggingface-pytorch-inference-2023-12-13-03-31-22-982'

In [50]:
data = {
   "inputs": "In the midst of a bustling city, vibrant colors of street markets painted the scene. A melodic hum of conversations blended with the rhythmic sounds of footsteps on cobblestone streets. As the sun dipped below the horizon, city lights illuminated towering skyscrapers, creating a mesmerizing nocturnal panorama. Cafés bustled with laughter, while the aroma of diverse cuisines filled the air. In a nearby park, the tranquility of nature provided a respite. Underneath a starlit sky, people shared stories and dreams. Each moment was a tapestry woven with threads of connection, creating a mosaic of experiences that echoed the heartbeat of urban life."
}

# request
predictor.predict(data)

[{'generated_text': ' In the midst of a bustling city, vibrant colors of street markets painted the scene . Cafés bustled with laughter, while the aroma of diverse cuisines filled the air . In a nearby park, the tranquility of nature provided a respite . Underneath a starlit sky, people shared stories and dreams .'}]

In [38]:
role = get_execution_role()
print(role)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
arn:aws:iam::593489176864:role/service-role/AmazonSageMaker-ExecutionRole-20231212T202053


In [44]:
predictor.endpoint_name

'huggingface-pytorch-inference-2023-12-13-04-46-52-847'